In [8]:
import os
from PIL import Image
#import tensorflow as tf
#from tensorflow import keras
import keras
#from tensorflow.keras import layers
from keras import layers
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.style.use("ggplot")
%matplotlib inline
import tensorflow as tf
tf.test.is_gpu_available(),tf.__version__,keras.__version__

(True, '2.0.0', '2.2.4')

In [7]:
tf.__path__

['E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow_core\\python\\keras\\api\\_v1',
 'E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow_estimator\\python\\estimator\\api\\_v1',
 'E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow_core\\_api\\v2\\compat\\v1']

mm = np.array(Image.open('E:\\Study\\竞赛\\识别验证码\\train\\1.jpg'))
mm =np.swapaxes(pp,1,2)
mm = mm.reshape(1,4,30,40,3)
mm =np.swapaxes(mm,2,3)
Image.fromarray(mm[0,3])

In [2]:
base_path = 'E:\Study\竞赛\识别验证码'

def read_images(root=base_path, train=True):
    def fun(i):
        if ord(i)< 65:
            return ord(i)- 48
        elif ord(i)>=97:
            return ord(i)-71+10 
        else:
            return ord(i) - 65 + 10 
    csv_fname = os.path.join(root, 'train/train_label.csv')
    csv_data = pd.read_csv(csv_fname, usecols=['ID', 'label'])
    images,labels = csv_data.to_numpy()[:,0].tolist(),csv_data.to_numpy()[:,1].tolist()
    images = [os.path.join(root, 'train\\'+i) for i in images]
    images = np.array([np.array(Image.open(os.path.join(base_path, i))) for i in images])
    labels = np.array([[fun(i) for i in i] for i in labels])
    return images, labels

In [3]:
images, labels = read_images() #5000 * 40 * 120 * 3 ;  5000 * 4 
#labels = np.array([tf.keras.utils.to_categorical(i, 62) for i in labels]) #5000 * 4 * 62
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(30)

In [11]:
inputs = layers.Input(shape=(40, 120, 3), name='img')

h1 = layers.Conv2D(32, 3, activation='relu', padding='same',strides=1, kernel_initializer='glorot_normal',use_bias=False)(inputs)
h1 = layers.BatchNormalization()(h1)
h1 = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal',use_bias=False)(h1)
block1_out = layers.BatchNormalization()(h1)
#block1_out = layers.MaxPooling2D(2)(h1) # N * 20 * 60 * 64

h2 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(block1_out)
h2 = layers.BatchNormalization()(h2)
h2 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(h2)
h2 = layers.BatchNormalization()(h2)
block2_out = layers.ReLU()(layers.add([h2, block1_out])) # N * 20 * 60 * 64

h3 = layers.Conv2D(64, 3, activation='relu',strides=2, padding='same',kernel_initializer='glorot_normal',use_bias=False)(block2_out)
h3 = layers.BatchNormalization()(h3)
#h3 = layers.MaxPooling2D(2)(h3)  
h3 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(h3)
h3 = layers.BatchNormalization()(h3)
block2_out = layers.Conv2D(64, 1, activation='relu', padding='same',strides=2,kernel_initializer='glorot_normal',use_bias=False)(block2_out)
block3_out = layers.ReLU()(layers.add([h3, block2_out])) # N * 10 * 30 * 64

h4 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(block3_out)
h4 = layers.BatchNormalization()(h4)
h4 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(h4)
h4 = layers.BatchNormalization()(h4)
block4_out = layers.ReLU()(layers.add([h4, block3_out])) # N * 10 * 30 * 64

h5 = layers.convolutional.ZeroPadding2D(padding=(2, 2), data_format=None)(block4_out)
h5 = layers.Conv2D(128, 3, activation='relu',strides=2, padding='valid',kernel_initializer='glorot_normal',use_bias=False)(h5)
h5 = layers.BatchNormalization()(h5)
#h5 = layers.MaxPooling2D(2)(h5)
h5 = layers.Conv2D(128, 3, activation='relu',padding='same',kernel_initializer='glorot_normal',use_bias=False)(h5) 
h5 = layers.BatchNormalization()(h5)
block4_out = layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format=None)(block4_out)
block4_out = layers.Conv2D(128, 1, activation='relu', padding='same',strides=2,kernel_initializer='glorot_normal',use_bias=False)(block4_out)
block5_out = layers.ReLU()(layers.add([h5, block4_out])) # N * 6 * 16 * 128

h6 = layers.Conv2D(256, 3, activation='relu',padding='same',kernel_initializer='glorot_normal',use_bias=False)(block5_out)
h6 = layers.BatchNormalization()(h6)
h6 = layers.Conv2D(256, 3, activation='relu',padding='same',kernel_initializer='glorot_normal',use_bias=False)(h6) 
h6 = layers.BatchNormalization()(h6)
block5_out = layers.Conv2D(256, 1, activation='relu', padding='same',strides=1,kernel_initializer='glorot_normal',use_bias=False)(block4_out)
block6_out = layers.ReLU()(layers.add([h6, block5_out]))# N * 6 * 16 * 128

out = keras.layers.Lambda(lambda x:keras.backend.permute_dimensions(x,(0,2,1,3)))(block6_out)
out = layers.Reshape((4,6,4,256))(out)
out = layers.TimeDistributed(layers.Conv2D(256, 1, activation='relu', strides=1,kernel_initializer='glorot_normal',use_bias=False))(out)
out = keras.layers.Lambda(lambda x:keras.backend.permute_dimensions(x,(0,1,3,2,4)))(out)
out = layers.Reshape((4,4*6*256))(out)
out = layers.TimeDistributed(layers.Dense(256, activation='relu', name='fc1'))(out) # N * 4 * 256
out = layers.TimeDistributed(layers.Dropout(0.5))(out)
out = layers.TimeDistributed(layers.Dense(64, activation='linear', name='dense_class'))(out)  # N * 4 * 64


model = keras.Model(inputs,out)
#keras.utils.plot_model(model, show_shapes=True)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [116]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 40, 120, 3)   0                                            
__________________________________________________________________________________________________
conv2d_558 (Conv2D)             (None, 40, 120, 32)  864         img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_448 (BatchN (None, 40, 120, 32)  128         conv2d_558[0][0]                 
__________________________________________________________________________________________________
conv2d_559 (Conv2D)             (None, 20, 60, 64)   18432       batch_normalization_448[0][0]    
__________________________________________________________________________________________________
batch_norm

In [117]:
optimizer = keras.optimizers.RMSprop(lr=0.0001)
model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

W1105 18:30:24.062805  7536 deprecation_wrapper.py:119] From E:\Study\Anaconda\envs\keras-gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [128]:
labels.shape

(5000, 4, 62)

In [ ]:
history_n = model.fit(images, labels.reshape(5000,4,1) , epochs=40, batch_size=30, validation_split=0.2,)

W1105 18:39:53.374544  7536 deprecation.py:323] From E:\Study\Anaconda\envs\keras-gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 4000 samples, validate on 1000 samples
Epoch 1/40
